In [88]:
import os
os.chdir("/Users/arpitverma/kidney-disease-Classification-DeepLearning")

In [89]:
%pwd

'/Users/arpitverma/kidney-disease-Classification-DeepLearning'

In [90]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_early_stopping_patience: int
    params_reduce_lr_patience: int
    params_reduce_lr_factor: float
    params_min_lr: float

In [91]:
from src.kidneyClassifier.constants import *
from src.kidneyClassifier.utils.common import read_yaml, create_directories

import tensorflow as tf

In [92]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training.training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_early_stopping_patience=params.EARLY_STOPPING_PATIENCE,
            params_reduce_lr_patience=params.REDUCE_LR_PATIENCE,
            params_reduce_lr_factor=params.REDUCE_LR_FACTOR,
            params_min_lr=params.MIN_LR
        )

        return training_config
    


 

In [93]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    
    def train_valid_generator(self):
        """
        Creates training and validation generators from a single directory.
        Uses validation_split=0.20 to automatically split data into 80% train and 20% validation.
        """
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20  # 20% for validation, 80% for training
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="categorical"
        )

        # Validation generator (no augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",  # Uses 20% of data
            shuffle=False,
            **dataflow_kwargs
        )

        # Training generator (with augmentation if enabled)
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",  # Uses 80% of data
            shuffle=True,
            **dataflow_kwargs
        )


    @staticmethod
    def compute_class_weights(generator):
        """Compute class weights for imbalanced datasets"""
        labels = generator.classes
        class_weights = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(labels),
            y=labels
        )
        class_weight_dict = dict(enumerate(class_weights))
        return class_weight_dict


    def get_callbacks(self):
        """Create callbacks for training"""
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=self.config.params_early_stopping_patience,
            restore_best_weights=True,
            verbose=1
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=self.config.params_reduce_lr_factor,
            patience=self.config.params_reduce_lr_patience,
            min_lr=self.config.params_min_lr,
            verbose=1
        )

        return [early_stopping, reduce_lr]


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Compute class weights
        class_weight_dict = self.compute_class_weights(self.train_generator)
        
        # Get callbacks
        callbacks = self.get_callbacks()

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            class_weight=class_weight_dict,
            callbacks=callbacks
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [85]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2025-11-08 16:40:43,517: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-08 16:40:43,521: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-08 16:40:43,522: INFO: common: created directory at: artifacts]
[2025-11-08 16:40:43,523: INFO: common: created directory at: artifacts/training]
[2025-11-08 16:40:44,567: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 2489 images belonging to 1 classes.
Found 9957 images belonging to 1 classes.


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 4)